In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline

import warnings;
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel('all_data.xlsx')
df.head()

,book_title,Book_series,book_series_url,book_image,book_image_url,book_rating,book_author,author_url,genre,reviewer_name,reviewer_url,reviewer_image,reviewer_image_url,review,ID
0,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",emma,https://www.goodreads.com/user/show/32879029-emma,https://images.gr-assets.com/users/1520568130p...,https://www.goodreads.com/user/show/32879029-emma,202.\nThatâ€™s the number of times that the wo...,1
1,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Em (RunawayWithDreamthieves),https://www.goodreads.com/user/show/42130592-e...,https://images.gr-assets.com/users/1538507799p...,https://www.goodreads.com/user/show/42130592-e...,I need a black silk robe and a balcony to wear...,2
2,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Cait â€¢ A Page with a View,https://www.goodreads.com/user/show/47412532-c...,https://images.gr-assets.com/users/1523134490p...,https://www.goodreads.com/user/show/47412532-c...,THIS BOOK TOTALLY WRECKED ME. In the best poss...,3
3,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Natalie Monroe,https://www.goodreads.com/user/show/6688207-na...,https://images.gr-assets.com/users/1339730817p...,https://www.goodreads.com/user/show/6688207-na...,"3.75 stars\n""Rhys stepped off the stairs and t...",4
4,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Emily May,https://www.goodreads.com/user/show/4622890-em...,https://images.gr-assets.com/users/1528390751p...,https://www.goodreads.com/user/show/4622890-em...,I wish so much that I could tell you I loved t...,5


In [3]:
df.shape

(21559, 15)

In [11]:
df  = pd.DataFrame(df[['book_title','book_rating','genre','review']])

## Treat Missing Data

In [12]:
# check for missing values
df.isnull().sum()

book_title     0
book_rating    0
genre          0
review         0
dtype: int64

In [13]:
# remove missing values
df.dropna(inplace=True)

# again check missing values
df.isnull().sum()

book_title     0
book_rating    0
genre          0
review         0
dtype: int64

## Data Preparation

In [14]:
df['book_title']= df['book_title'].astype(str)

In [9]:
book = df["ID"].unique().tolist()
len(book)

KeyError: 'ID'

In [ ]:
# shuffle customer ID's
random.shuffle(book)

# extract 90% of customer ID's
book_train = [book[i] for i in range(round(0.9*len(book)))]

# split data into train and validation set
train_df = df[df['ID'].isin(book_train)]
validation_df = df[~df['ID'].isin(book_train)]

In [ ]:
# list to capture purchase history of the customers
book_train = []

# populate the list with the product codes
for i in tqdm(book_train):
    temp = train_df[train_df["ID"] == i]["book_title"].tolist()
    book_train.append(temp)

In [ ]:
# list to capture purchase history of the customers
book_val = []

# populate the list with the product codes
for i in tqdm(validation_df['ID'].unique()):
    temp = validation_df[validation_df["ID"] == i]["book_title"].tolist()
    book_val.append(temp)

## Build Word2Vec Embeddings for Books

In [ ]:
# train word2vec model
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model.build_vocab(book_train, progress_per=200)

model.train(book_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)